## Object (somewhat) Localisation
### Import modules

In [2]:
import os
import numpy as np
import cv2 # Extract images from video
import tensorflow as tf # Tensorflow stuff

### Training Data

In [3]:
try:
    if not os.path.exists('training'):
        os.makedirs('training')
except OSError:
    print('Error creating directory - training')

### Import sample.mp4

In [4]:
try:
    if not os.path.exists('data'):
        os.makedirs('data')
except OSError:
    print('Error creating directory - data')

vid = cv2.VideoCapture("./data/sample.mp4")

### Model

In [13]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [12]:
# PRETRAINED RESNET MODEL WITHOUT FC LAYER
base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze trainable layers except for last few layers
base_model.trainable = False

output = base_model.output
output = tf.keras.layers.Flatten() (output)

# Add new FC Layers to the output that can predict the XY coordinate and prediction
output = tf.keras.layers.Dense(128, activation="relu") (output)
output = tf.keras.layers.Dense(64, activation="relu") (output)
output = tf.keras.layers.Dense(32, activation="relu") (output)
output = tf.keras.layers.Dense(3, activation="sigmoid") (output) # sigmoid because binary imageclassification, 3 units for XY coordinate and prediction

model = tf.keras.models.Model(inputs=base_model.input, outputs=output)
# model.summary()

In [ ]:
# MODEL PARAMETERS
EPOCHS = 5
BATCH_SIZE = 32
OPTIMIZER = tf.optimizers.Adam(learning_rate=0.001)
LOSS_FUNC = tf.keras.losses.mean_squared_error # Use MSE because I want to check how close the predictions are to the actual values

# Callbacks - Stop early if val_loss starts increasing suddenly
CALLBACKS = [
    tf.keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=3, mode='min',
                restore_best_weights=True
            )
        ]

# Initialize optimizer, compile and start fine tuning the model
model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNC)

# Fit the model
model.fit()